In [1]:
from collections import defaultdict


def load_data(path):
    with open(path) as f:
        data = f.read()
    return data


def convert_disk_map_to_individual_blocks(disk_map):
    individual_blocks = []
    for id_number, length in enumerate(disk_map):
        for i in range(int(length)):
            if id_number & 1:
                individual_blocks.append(-1)
            else:
                individual_blocks.append(id_number // 2)
    return individual_blocks


def move_blocks_to_left(blocks):
    sorted_blocks = blocks.copy()
    left_id = 0
    right_id = len(sorted_blocks)-1
    while left_id < right_id:
        left_value = sorted_blocks[left_id]
        right_value = sorted_blocks[right_id]
        if left_value != -1:
            left_id += 1
            continue
        else:
            if right_value != -1:
                sorted_blocks[left_id] = right_value
                sorted_blocks[right_id] = -1
            right_id -= 1
    return sorted_blocks


def group_elements(elements):
    elements = elements.copy()
    previous_element = elements[0]
    grouped_elements = [[elements[0]]]
    n_group = 0
    for element in elements[1:]:
        if len(grouped_elements) <= n_group:
            grouped_elements.append([])
        if element - previous_element == 1:
            grouped_elements[n_group].append(element)
        else:
            grouped_elements.append([])
            n_group += 1
            grouped_elements[n_group].append(element)
        previous_element = element
    return grouped_elements


def move_files_to_left(blocks):
    dd = defaultdict(list)
    for i, v in enumerate(blocks):
        dd[v].append(i)
    len_files = len(dd) - 1
    free_spaces = group_elements(dd[-1])
    n_moved_files = 0
    moved_files = []
    for i in range(len_files):
        file_id = len_files - i - 1
        file_indices = dd[file_id]
        for j, free_space_group in enumerate(free_spaces):
            if len(free_space_group) >= len(file_indices):
                if free_space_group[0] < file_indices[0]:
                    moved_files.append([])
                    moved_files[n_moved_files].append(free_space_group[:len(file_indices)])
                    moved_files[n_moved_files].append(file_indices)
                    n_moved_files += 1
                    free_spaces[j] = free_space_group[len(file_indices):]
                    dd.pop(file_id)
                    break
    sorted_blocks = blocks.copy()
    for src, dst in moved_files:
        for i in range(len(src)):
            sorted_blocks[src[i]] = blocks[dst[i]]
            sorted_blocks[dst[i]] = -1
    return sorted_blocks


def get_checksum(sorted_blocks):
    checksum = 0
    for i, v in enumerate(sorted_blocks):
        if v == -1:
            continue
        else:
            checksum += i * v
    return checksum


def solve_part_1(data):
    blocks = convert_disk_map_to_individual_blocks(data)
    sorted_blocks = move_blocks_to_left(blocks)
    return get_checksum(sorted_blocks)


def solve_part_2(data):
    blocks = convert_disk_map_to_individual_blocks(data)
    sorted_blocks = move_files_to_left(blocks)
    return get_checksum(sorted_blocks)


data = load_data('input.txt')
print(solve_part_1(data))
print(solve_part_2(data))

6332189866718
6353648390778
